# Import Data

In [3]:
import pandas as pd

green_line_info   = pd.read_csv('./TrackLayoutGreen.csv')
red_line_info     = pd.read_csv('./TrackLayoutRed.csv')
green_line_layout = pd.read_csv('./green_line.csv')
red_line_layout   = pd.read_csv('./red_line.csv')
green_line_layout

,Section,Block,Oneway,Infra,Prev,Next,Branch R,Branch F,Station
0,A,1,rev,NaN,13,2,NaN,NaN,NaN
1,A,2,rev,NaN,1,3,NaN,NaN,Pioneer:R
2,A,3,rev,NaN,2,4,NaN,NaN,NaN
3,B,4,rev,NaN,3,5,NaN,NaN,NaN
4,B,5,rev,NaN,4,6,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
145,X,146,fwd,NaN,145,147,NaN,NaN,NaN
146,Y,147,fwd,NaN,146,148,NaN,NaN,NaN
147,Y,148,fwd,NaN,147,149,NaN,NaN,NaN
148,Y,149,fwd,NaN,148,150,NaN,NaN,NaN


,Section,Block,Length,Grade,Speed,Oneway,Infra,Prev,Next,Branch R,Branch F,Station
0,A,1,50.0,0.5,40,NaN,NaN,16,2,NaN,NaN,NaN
1,A,2,50.0,1.0,40,NaN,NaN,1,3,NaN,NaN,NaN
2,A,3,50.0,1.5,40,NaN,NaN,2,4,NaN,NaN,NaN
3,B,4,50.0,2.0,40,NaN,NaN,3,5,NaN,NaN,NaN
4,B,5,50.0,1.5,40,NaN,NaN,4,6,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
71,R,72,50.0,0.0,55,NaN,1.0,33,73,NaN,NaN,NaN
72,S,73,50.0,0.0,55,NaN,1.0,72,74,NaN,NaN,NaN
73,S,74,50.0,0.0,55,NaN,1.0,73,75,NaN,NaN,NaN
74,S,75,50.0,0.0,55,NaN,1.0,74,76,NaN,NaN,NaN


In [4]:
from math import isnan
import numpy as np
type(green_line_layout['Oneway'][0]) == np.float64

False

# Green Line

## Display Graph

In [13]:
import json

cyto = {
    'nodes': [],
    'edges': []
}

# Position of section starts
section_positions = {
    'A': (485,  23),
    'B': (544,  96),
    'C': (708,  60),
    'D': (485,  23),
    'E': (331,  26),
    'F': (234, 118),
    'G': (236, 269),
    'H': (236, 347),
    'I': (315, 415),
    'J': (709, 415),
    'K': (888, 518),
    'L': (888, 804),
    'M': (759, 956),
    'N': (527, 956),
    'O': (322, 956),
    'P': (198, 956),
    'Q': (165, 824),
    'R': (527, 956),
    'S': (580, 925),
    'T': (690, 925),
    'U': (833, 775),
    'V': (833, 550),
    'W': (680, 445),
    'X': (265, 445),
    'Y': (182, 404),
    'Z': (182, 319)
}

def transform_section(section, dx, dy):
    x, y = section_positions[section]
    x += dx
    y += dy
    section_positions[section] = (x, y)

# Transform for widening the map, since W/I have sections are dense
for section in ['G', 'H', 'I', 'X', 'Y', 'Z']:
    transform_section(section, dx=-600, dy=0)

transform_section('G', dx=0, dy=-70)
transform_section('Z', dx=0, dy=-70)

for section in ['F', 'E', 'D', 'A', 'B', 'C']:
    transform_section(section, dx=-200, dy=0)
    
transform_section('F', dx=-100, dy=0)

for section in ['O', 'P', 'Q']:
    transform_section(section, dx=-400, dy=0)
    
transform_section('Q', dx=-100, dy=-200)

# Used for interpolating block points in sections
next_sections = {
    'A': 'B',
    'B': 'C',
    'C': 'D',
    'D': 'E',
    'E': 'F',
    'F': 'G',
    'G': 'H',
    'H': 'I',
    'I': 'J',
    'J': 'K',
    'K': 'L',
    'L': 'M',
    'M': 'N',
    'N': 'O',
    'O': 'P',
    'P': 'Q',
    'Q': 'O',
    'R': 'S',
    'S': 'T',
    'T': 'U',
    'U': 'V',
    'V': 'W',
    'W': 'X',
    'X': 'Y',
    'Y': 'Z',
    'Z': 'G'
}

In [66]:
def get_section_blocks(layout, section):
    return layout.query(f'`Section` == "{section}"')

def get_section_start_block_id(layout, section):
    all_section_blocks = get_section_blocks(layout, section)
    idx_min = all_section_blocks['Block'].idxmin()
    return all_section_blocks['Block'][idx_min]

# Interpolate a line and divide into n_divs divisions
def interpolate(c0, c1, n_divs) -> list[tuple[float, float]]:
    x0, y0 = c0
    x1, y1 = c1

    x_div = (x1 - x0) / n_divs;
    y_div = (y1 - y0) / n_divs;
    
    li = [(x0, y0)]
    
    x_now = x0
    y_now = y0
    for i in range(n_divs):
        x_now += x_div
        y_now += y_div
        
        li.append((x_now, y_now))
    
    return li

def sliding_window(elements, window_size):
    li = []

    if len(elements) <= window_size:
        return elements

    for i in range(len(elements) - (window_size - 1)):
        li.append(elements[i:i+window_size])

    return li

def gen_node(block_id, x, y):
    return {
        'data': {
            'id': block_id
        },
        'position': {
            'x': int(2*x),
            'y': int(2*y)
        }
    }

def gen_edge(block_id, from_node, to_node, block_object, block_info):
    edge = {
        'data': {
            'id': 'Block' + str(block_id),
            'section': block_object['Section'],
            'block_id': block_object['Block'],
            'length': float(block_info['Length'].iloc[0]),
            'grade': float(block_info['Grade'].iloc[0]),
            'speed_limit': float(block_info['Speed Limit'].iloc[0]),
            'elevation': float(block_info['Elevation'].iloc[0]),
            'block_name': block_object['Block'],
            'source': str(from_node),
            'target': str(to_node)
        }
    }
    
    return edge

# At forks there's overlap with the 'node with ID n leads to block n' thing
def resolve_overlap(block_id):
    if block_id == 13:
        return 1
    elif block_id == 101:
        return 77
    else:
        return block_id
    
def update_line(line, nodes, edges):
    with open('../TrackModel-display.json', 'r') as input:
        trackmodel_display = json.load(input)

    cyto = {
        'nodes': nodes,
        'edges': edges
    }
    trackmodel_display['lines'][line] = cyto

    with open('../TrackModel-display.json', 'w') as output:
        json.dump(trackmodel_display, output)

In [59]:
block_info['Length'].iloc[0]

35.0

In [67]:
nodes_seen = []
nodes = []

# -- Generate nodes --
# Node with ID n leads into block n
for section in section_positions.keys():
    next_section = next_sections[section]
    all_section_blocks = get_section_blocks(green_line_layout, section).reset_index()
    points = interpolate(
                section_positions[section],
                section_positions[next_section],
                len(all_section_blocks)
    )

    for index, block in all_section_blocks.iterrows():
        # Don't add if it's a fork/merge and its an overlap
        block_id = int(block['Block'])
        resolved_block_id = resolve_overlap(block_id)
        if(resolved_block_id not in nodes_seen):
            nodes_seen.append(resolved_block_id)
        else:
            continue

        x, y = points[index]
        node = gen_node(block['Block'], x, y)
        nodes.append(node)

# -- Generate edges --
edges = []
for index, block in green_line_layout.iterrows():
    block_id = int(block['Block'])
    prev_id = resolve_overlap(block['Prev'])
    next_id = resolve_overlap(block['Next'])
    
    block_info = green_line_info.query(f'`Block ID` == {block_id}')
    
    if(block['Oneway'] == 'fwd'):
        edge = gen_edge(block_id, resolve_overlap(block_id), next_id, block, block_info)
        edges.append(edge)
        continue
        
    if(block['Oneway'] == 'rev'):
        edge = gen_edge(block_id, next_id, resolve_overlap(block_id), block, block_info)
        edges.append(edge)
        continue
        
    # Being here -> it's probably bidirectional
    edge = gen_edge(f'{block_id}f', next_id, resolve_overlap(block_id), block, block_info)
    edges.append(edge)
    edge = gen_edge(f'{block_id}r', resolve_overlap(block_id), next_id, block, block_info)
    edges.append(edge)
    
update_line('green', nodes, edges)

,Line,Section,Block ID,Length,Grade,Speed Limit,Infrastructure,Station Side,Elevation,Cumulative Elevation,seconds to traverse block
0,Green,A,1,100.0,0.5,45,NaN,NaN,0.5,0.5,8.0


In [34]:
initial_block_idx = green_line['Block ID'].idxmin()
initial_block_id  = green_line['Block ID'][initial_block_idx]
final_block_idx   = green_line['Block ID'].idxmax()
final_block_id    = green_line['Block ID'][final_block_idx]

In [65]:
import json

cyto = {
    'nodes': [],
    'edges': []
}

# Section directions relative to starting position
section_directions = {
    'A': 'r',
    'B': 'r',
    'C': 'r',
    'D': 'b',
    'E': 'b',
    'F': 'b',
    'G': 'f',
    'H': 'f',
    'I': 'f',
    'J': 'f',
    'K': 'f',
    'L': 'f',
    'M': 'f',
    'N': 'b',
    'O': 'f',
    'P': 'f',
    'Q': 'f',
    'R': 'f',
    'S': 'f',
    'T': 'f',
    'U': 'f',
    'V': 'f',
    'W': 'f',
    'X': 'f',
    'Y': 'f',
    'Z': 'f'
}

def gen_node(section, block_id, x, y):
    return {
        'data': {
            'id': section + block_id,
            'node_name': section + block_id
        },
        'position': {
            'x': int(x),
            'y': int(y)
        }
    }

# Interpolate a line and spread n_points equidistant points
def interpolate(c0, c1, n_divs) -> list[tuple[float, float]]:
    x0, y0 = c0
    x1, y1 = c1

    x_div = (x1 - x0) / n_divs;
    y_div = (y1 - y0) / n_divs;
    
    li = [(x0, y0)]
    
    x_now = x0
    y_now = y0
    for i in range(n_divs):
        x_now += x_div
        y_now += y_div
        
        li.append((x_now, y_now))
    
    return li

def sliding_window(elements, window_size):
    li = []

    if len(elements) <= window_size:
        return elements

    for i in range(len(elements) - (window_size - 1)):
        li.append(elements[i:i+window_size])

    return li

In [59]:
sliding_window([1, 2, 3, 4, 5, 6], 2)

[[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]]

In [66]:
section = green_line['Section'][initial_block_idx]
block_id = str(initial_block_id - 1)

prev_section = 'ASDSAIDSAODIj'
for prev_block, next_block in sliding_window(list(green_line.iterrows()), 2):    
    current_section = block['Section']

    if(current_section != prev_section):
        x, y = section_positions[current_section]
        cyto['nodes'].append(gen_node(current_section, '0', x, y))
        prev_section = current_section

print(json.dumps(cyto))

TypeError: tuple indices must be integers or slices, not str